### Explore where initial tokens were sent
##### This notebook contains code that looks at the initial minting of a token and tracks where the tokens were sent

Assumption: the minting address is assumed to be the first one that sends the tokens to at least 2 other addresses

In [1]:
# !pip install pandas
# !pip install tqdm

In [2]:
import pandas as pd
import requests
from tqdm.notebook import tqdm
pd.set_option('display.float_format', lambda x: '%.1f' % x)

token_to_analyze = 'erdoge'

In [3]:
token_ids = {'kushti':'fbbaac7337d051c10fc3da0ccb864f4d32d40027551e1c3ea3ce361f39b91e40',
             'erdoge':'36aba4b4a97b65be491cf9f5ca57b5408b0da8d0194f30ec8330d1e8946161c1',
             'lunadog':'5a34d53ca483924b9a6aa0c771f11888881b516a8d1a9cdc535d063fe26d065e',
             'comet':'0cd8c9f416e5b1ca9f986a7f10a84191dfb85941619e49e53c0dc30ebf83324b'
             }

api_url = 'https://api.ergoplatform.com/api/v1/'
api_token = api_url+'boxes/byTokenId/'
api_token_info = api_url+'tokens/'
api_box = api_url+'boxes/'
api_transaction = api_url+'transactions/'
api_address_transactions = api_url + 'boxes/byAdress/'
page_limit = 100 #ask the api to return this many items in each "page"
page_max = 1000 #maximum pages to look through

def parse_transaction(tx_id,token_id):
    """get transaction with tx_id and return all
       output boxes that contain asset with token_id"""
    tx = requests.get(api_transaction+tx_id).json()
    outputs = tx['outputs']
    out_boxes = [i for i in outputs if token_id in [j['tokenId'] for j in i['assets']]]
    return out_boxes

def get_txs(addr):
    """Given an address, get all transactions made by that address.
    This can take some time if the address has made many txs
    """
    print(f"getting txs from addr: {addr}...")
    txs = pd.DataFrame()
    for p in range(0,page_max):
        items = requests.get(f"{api_url}addresses/{addr}/transactions?offset={str(p*page_limit)}&limit={str(page_limit)}").json()['items']
        items = pd.DataFrame(items)
        if len(items) == 0:
            break
        txs = pd.concat([txs,items])
    return txs

def amount_in_box(assets,token_id):
    token_assets = [i for i in assets if i['tokenId']==token_id]
    token_assets = token_assets[0]
    token_amount = token_assets['amount']
    return token_amount
        
token_name = token_to_analyze
token_id = token_ids[token_name]
token_info = requests.get(api_token_info+token_id).json()
total_supply = token_info['emissionAmount']
print(f"Token name: {token_name} -- emission amount: {total_supply} (decimals: {token_info['decimals']})")
box_0 = requests.get(api_box+token_info['boxId']).json() #the box containing the initial tokens
minting_addr = box_0['address'] #address that minted the tokens
tx_id_0 = box_0['spentTransactionId'] #1st transaction that spent the minted tokens
if tx_id_0 is None:
    print("Tokens were never sent from initial address")

tx_id = tx_id_0
while True: 
    """the minting address is assumed to be the first one 
    that sends the tokens to at least 2 other addresses.
    This loop updates the minting_addr if the whole supply was sent
    from the original address to another."""
    boxes = parse_transaction(tx_id,token_id)
    if len(boxes)==1:
        tx_id = boxes[0]['spentTransactionId']
        minting_addr = boxes[0]['address']
    else:
        break

minting_addr_txs = get_txs(minting_addr)
boxes_out = pd.DataFrame()
tx_ids = minting_addr_txs.id.values
print("scanning transcations from minting address")
for i in tqdm(range(len(tx_ids))):
    boxes_i = parse_transaction(tx_ids[i],token_id)
    boxes_i = [j for j in boxes_i if j['address']!=minting_addr]
    if len(boxes_i)==0:
        continue
    boxes_df = pd.DataFrame() #temporoary dataframe
    boxes_df['address'] = [j['address'] for j in boxes_i]
    boxes_df['amount'] = [amount_in_box(j['assets'],token_id) for j in boxes_i]
    boxes_df['block'] = [j['creationHeight'] for j in boxes_i]
    boxes_df['percentOfTotal'] = boxes_df['amount'].apply(lambda x: 100*x/token_info['emissionAmount'])
    boxes_out = pd.concat([boxes_out,boxes_df])

boxes_out = boxes_out.sort_values(by='block') #oldest first
boxes_out['amount'] = boxes_out['amount'].apply(lambda x: x*(10**(-token_info['decimals'])))


Token name: erdoge -- emission amount: 500000 (decimals: 0)
getting txs from addr: 9h2h2JsXDJwnnffZf7FDzigTdLqAUamzxZB2uoUjNiz7BBf3mXd...
scanning transcations from minting address


  0%|          | 0/127 [00:00<?, ?it/s]

#### Because we counted only outflows from the minting address, we may have counted more than the total supply, if there was some inflow of the tokens back to the minting address.

In [4]:
boxes_out

,address,amount,block,percentOfTotal
0,9hYyTkVtRSQy4TEwimVLiG6TsmYtDVc66xmG15Rqk6SnYR...,50000,537082,10.0
0,9gdYyHmNqiCTM6hU4EA11CNvcvwgzZT9JRW5qJtKcT7vZC...,50000,537156,10.0
0,9hj3zqd2J21yYWQQCK9oSdw79iJTKKZAdFBAuVjzXGx8PP...,25000,537340,5.0
0,9hJ86eKFLugJvctbQGQPLvNhzV2SJrFZzm8Nurgsx4YLYH...,25000,537345,5.0
0,9eaytg9fjXUCJEUEHNKvr5hDiBpJ9AcZGZ2N8YLcSQyVbj...,10000,537687,2.0
0,9gci45ZwRHcv6fgZUmTrqsoQVZqmWDRA728ymaTeE7aJn2...,1,537716,0.0
0,9h9McVPsdANFdhX6JRNe2x99iw6cXC9Zfo3vLMUb9WiBeD...,25000,538362,5.0
0,9iLoGNUNfyXuFYSdWxbDTtjWPXhYWdDKssSf46f5JHKQbb...,10000,538383,2.0
0,9hxbycmWbqgRqfxubbi5YgYpVZ6MKY1oX4XoFwgQXbs1Mg...,304999,538394,61.0
